# Assign DS model

## Set up example environment

In [1]:
%run helpers/example_setup.ipynb

db, cell, layout variables set to the default from edb design.


## Get Layers

In [2]:
lc = layout.layer_collection
layers = lc.get_layers()
for i in layers:
    if i.is_stackup_layer:
        name = i.name
        layer_type = i.type.name
        material = i.get_material()
        print(f"Layer '{name}', type '{layer_type}', material '{i.get_material()}'")

Layer 's1', type 'SIGNAL_LAYER', material 'copper'
Layer 'd1', type 'DIELECTRIC_LAYER', material 'FR4_epoxy'
Layer 's2', type 'SIGNAL_LAYER', material 'copper'
Layer 'd2', type 'DIELECTRIC_LAYER', material 'FR4_epoxy'
Layer 's3', type 'SIGNAL_LAYER', material 'copper'


## Assign DS model to dielectric layers

In [3]:
for i in layers:
    if i.is_stackup_layer:
        name = i.name
        layer_type = i.type.name
        material = i.get_material()
       
        if 'DIELECTRIC_LAYER' in str(layer_type):
            matmodel = MaterialDef.find_by_name(db,material)
            matmodel.dielectric_material_model= DjordjecvicSarkarModel.create()
            print(f"Assigned DS model to Layer '{name}'")

Assigned DS model to Layer 'd1'
Assigned DS model to Layer 'd2'


## Tear down example environment

In [4]:
%run helpers/example_teardown.ipynb